#### 【 교차검증 (Cross Validation) - cross_val_score()/cross_validate() 】

- 적은 데이터셋으로 안정적이고 신뢰성있는 모델 평가를 위한 방법
- 학습 데이터셋을 K개 분할 후 매번 다른 데이터로 검증 진행
- 교차검증 후 모델의 일반화 성능으로 여김
- sklearn.model_selection 서브모듈에 존재하는 함수들
    * cross_val_score()/ cross_val_predict() : cv만큼에 대한 성능결과, 예측결과 반환
    * cross_validate() : 다양한 정보들 반환. 많이 사용됨!!

[1] 모듈 로딩 및 데이터 준비<hr>

In [8]:
## [1-1] 모듈 로딩
##-> 기본 모듈
import numpy as np
import pandas as pd

##-> ML 관련 모듈
from sklearn.model_selection import cross_val_score, cross_validate       ## 교차검증용
from sklearn.neighbors import KNeighborsClassifier                        ## 학습 알고리즘

In [3]:
## [1-2] 데이터 준비
DATA_FILE = '../Data/iris.csv'

irisDF = pd.read_csv(DATA_FILE)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 전처리 및 학습 준비 : 시간때문에 미진행 => 개인별로 진행 <hr>

In [4]:
## [2-1] 품종컬럼 자료형 변환
pd.options.mode.copy_on_write = True

irisDF.variety = irisDF.variety.astype('category')
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   sepal.length  150 non-null    float64 
 1   sepal.width   150 non-null    float64 
 2   petal.length  150 non-null    float64 
 3   petal.width   150 non-null    float64 
 4   variety       150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [5]:
## [2-2] 피쳐와 타겟분리
featureDF = irisDF[ irisDF.columns[:-1] ]
targetSR  = irisDF [ irisDF.columns[-1] ]

print(f'featureDF : {featureDF.shape},  targetSR : {targetSR.shape}')

featureDF : (150, 4),  targetSR : (150,)


In [ ]:
## [2-3] 수치형 컬럼 => 학습 알고리즘에 따라 적용
##                 => KNN 알고리즘은 거리측정 기반으로 스케일링 필요함!!


[3] 교차검증 <hr>

In [ ]:
## ==========================================================
## cross_validate() 함수 : 전달된 데이터에 따라서 자동으로 
##                         KFold, StratifiedKFold 설정
## - 필수 매개변수
##   estimator          : 모델 인스턴스
##   cv                 : 기본 5 또는 KFold, StratifiedKFold 인스턴스
##   return_train_score : 학습용 데이터셋 성능 반환여부 설정
##   return_estiator    : 학습 모델 인스턴스 
## ==========================================================
## 모델 인스턴스 생성
columns_  = ['fit_time', 'score_time', 'test_score', 'train_score', 'neighbors']
resultDF  = pd.DataFrame( columns=columns_ )

for neighbors in range(1, 21):
    kModel = KNeighborsClassifier(n_neighbors=neighbors)

    ## 함수 호출
    resultDict = cross_validate( kModel,
                                 featureDF,
                                 targetSR,
                                 return_train_score=True,
                                 cv=3)
    
    ret = [ resultDict[k].mean().item()  for k in resultDict.keys() ]
    ret.append(neighbors)
    resultDF.loc[resultDF.shape[0]] = ret


In [36]:
resultDF['diff'] = abs(resultDF['test_score'] - resultDF['train_score'])
resultDF

,fit_time,score_time,test_score,train_score,neighbors,diff
0,0.001501,0.001375,0.960000,1.000000,1.0,0.040000
1,0.001532,0.001683,0.946667,0.976667,2.0,0.030000
2,0.001000,0.001000,0.973333,0.963333,3.0,0.010000
3,0.001000,0.001000,0.986667,0.960000,4.0,0.026667
4,0.001000,0.001333,0.980000,0.970000,5.0,0.010000
5,0.000666,0.001333,0.973333,0.966667,6.0,0.006667
6,0.001000,0.001667,0.973333,0.970000,7.0,0.003333
7,0.000667,0.001669,0.966667,0.966667,8.0,0.000000
8,0.001001,0.001998,0.973333,0.970000,9.0,0.003333
9,0.000672,0.001665,0.966667,0.973333,10.0,0.006667
